## Imports ##

In [443]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif, RFECV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from xgboost import XGBClassifier

## Functions ##

In [444]:
def load_data(data_path):
    df = pd.read_csv(data_path)
    return df


def preprocess_data(df):
    # Convert 'color' column to binary representation
    df["color"] = df["color"].map({"red": 0, "green": 1})

    df["color_change"] = df["color"].diff().ne(0).astype(int)
    df["color_change"].fillna(0, inplace=True)

    # Fill NaNs in specified columns with 0
    df["PSARl_0.01_0.1"].fillna(0, inplace=True)
    df["PSARs_0.01_0.1"].fillna(0, inplace=True)
    df["ICS_15"].fillna(0, inplace=True)

    # Diff features

    df["volume_diff"] = df["volume"].diff()
    df["turnover_diff"] = df["turnover"].diff()
    df["close_open"] = df["close"] - df["open"]
    df["open_close"] = df["open"] - df["close"]
    df["high_low"] = df["high"] - df["low"]
    df["close_change"] = df["close"].diff()
    df["open_change"] = df["open"].diff()
    df["h_diff"] = df["high"] - df["high"].shift(1)
    df["l_diff"] = df["low"] - df["low"].shift(1)
    df["hl_shift"] = df["high"] - df["low"].shift(1)
    df["high_pct"] = df["high"].pct_change()
    df["low_pct"] = df["low"].pct_change()
    df["close_open_lag"] = df["close_open"].shift(1)
    df["high_low_lag"] = df["high_low"].shift(1)

    return df


def scale_data(df):
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df_scaled

## Globals ##

In [445]:
# # Set display options to show all rows and columns
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Load the data
data_path = (
    "../../../data/kc/btc/heiken_ashi/with_trade_indicators/raw/kc_btc_12min_ha_ti.csv"
)

# List of features to drop
features_to_drop = [
    ########
    # Univariate Feature Selection Features #
    #######
    "cmf",
    "MACDs_6_13_5_6_13_5",
    "RVI_10",
    "BBL_5_2.0_5",
    "STOCHk_14_3_3",
    "STOCHk_14_3_3_7_3_3",
    "STOCHk_14_3_3_10_3_3",
    "MACDh_6_13_5_6_13_5",
    "MACDh_12_26_9",
    "BBL_10_2.0_10",
    "R3",
    "R2",
    "R1",
    "open",
    "BBL_15_2.0_15",
    "SMA_5",
    "BBM_5_2.0_5",
    "PP",
    "BBL_20_2.0_20",
    "EMA_5",
    "ITS_5",
    "SMA_10",
    "BBM_10_2.0_10",
    "EMA_10",
    "S1",
    "SMA_20",
    "EMA_2",
    "BBM_15_2.0_15",
    "S2",
    "BBM_20_2.0_20",
    "IKS_15",
    "close",
    "PSARaf_0.01_0.1",
    "S3",
    "MACD_6_13_5_6_13_5",
    "ISA_5",
    "ISB_15",
    "BBU_15_2.0_15",
    "BBU_10_2.0_10",
    "BBB_15_2.0_15",
    "obv",
    "BBU_5_2.0_5",
    "high",
    "STOCHd_14_3_3",
    "STOCHd_14_3_3_7_3_3",
    "STOCHd_14_3_3_10_3_3",
    "TRIXs_10_5",
    "MACD_12_26_9",
    "PSARs_0.01_0.1",
    "TRIXs_18_9",
    "ICS_15",
    "ROC_10",
    "BBB_20_2.0_20",
    "bollinger_bandwidth",
    "RVI_15",
    "MACDs_12_26_9",
    "PSARl_0.01_0.1",
    "TRIX_18_9",
    "TRIXs_12_6",
    "ROC_5",
    "BBU_20_2.0_20",
    "low",
    "TRIX_12_6",
    "ROC_14",
]

## Preprocessing

In [446]:
df = load_data(data_path)

# Preprocess the data
df = preprocess_data(df)

# Drop the specified features
df = df.drop(columns=features_to_drop, errors="ignore")

# Prepare TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3)

## Scale Data

In [447]:
# Forward Fill
df.ffill(inplace=True)

# Backward Fill
df.bfill(inplace=True)

# Re-scale the data to include the new feature
df_scaled = scale_data(df)

# print(df.isna().sum())

## Ablation ##

In [448]:
# Define initial features for ablation process
ablation_features = [
    "cmf",
    "MACDs_6_13_5_6_13_5",
    "RVI_10",
    "BBL_5_2.0_5",
    "STOCHk_14_3_3",
    "STOCHk_14_3_3_7_3_3",
    "STOCHk_14_3_3_10_3_3",
    "MACDh_6_13_5_6_13_5",
    "MACDh_12_26_9",
    "BBL_10_2.0_10",
    "R3",
    "R2",
    "o_shift",
    "R1",
    "open",
    "BBL_15_2.0_15",
    "SMA_5",
    "BBM_5_2.0_5",
    "c_shift",
    "PP",
    "BBL_20_2.0_20",
    "EMA_5",
    "ITS_5",
    "SMA_10",
    "BBM_10_2.0_10",
    "EMA_10",
    "S1",
    "SMA_20",
    "EMA_2",
    "BBM_15_2.0_15",
    "S2",
    "BBM_20_2.0_20",
    "IKS_15",
    "close",
    "PSARaf_0.01_0.1",
    "S3",
    "MACD_6_13_5_6_13_5",
    "ISA_5",
    "ISB_15",
    "BBU_15_2.0_15",
    "BBU_10_2.0_10",
    "BBB_15_2.0_15",
    "obv",
    "BBU_5_2.0_5",
    "high",
    "STOCHd_14_3_3",
    "STOCHd_14_3_3_7_3_3",
    "STOCHd_14_3_3_10_3_3",
    "TRIXs_10_5",
    "MACD_12_26_9",
    "PSARs_0.01_0.1",
    "TRIXs_18_9",
    "ICS_15",
    "ROC_10",
    "BBB_20_2.0_20",
    "bollinger_bandwidth",
    "RVI_15",
    "MACDs_12_26_9",
    "PSARl_0.01_0.1",
    "TRIX_18_9",
    "volume_diff_lag",
    "TRIXs_12_6",
    "ROC_5",
    "BBU_20_2.0_20",
    "low",
    "TRIX_12_6",
    "ROC_14",
]

# All the features in the data
all_features = list(df.columns)
all_features.remove("color_change")  # we remove the target variable

log_reg_scores = {}
rf_scores = {}

for feature in ablation_features:
    # Remove one feature from the all features list
    current_features = [f for f in all_features if f != feature]

    # Prepare your features and target
    X = df[current_features]
    y = df["color_change"]

    # Logistic Regression
    log_reg = LogisticRegression(random_state=42, max_iter=500)

    # Cross-validation
    cv_scores = cross_val_score(log_reg, X, y, cv=tscv, scoring="roc_auc")
    log_reg_scores[feature] = np.mean(cv_scores)

# Print logistic regression scores sorted by score
print("Logistic Regression Scores")
for feature, score in sorted(
    log_reg_scores.items(), key=lambda item: item[1], reverse=True
):
    print(f"CV ROC AUC score without {feature}: {score}")

for feature in ablation_features:
    # Remove one feature from the all features list
    current_features = [f for f in all_features if f != feature]

    # Prepare your features and target
    X = df[current_features]
    y = df["color_change"]

    # Random Forest
    rf = RandomForestClassifier(n_estimators=100, random_state=42)

    # Cross-validation
    cv_scores = cross_val_score(rf, X, y, cv=tscv, scoring="roc_auc")
    rf_scores[feature] = np.mean(cv_scores)

# Print random forest scores sorted by score
print("\nRandom Forest Scores")
for feature, score in sorted(rf_scores.items(), key=lambda item: item[1], reverse=True):
    print(f"CV ROC AUC score without {feature}: {score}")

Logistic Regression Scores
CV ROC AUC score without cmf: 0.5765177753422486
CV ROC AUC score without MACDs_6_13_5_6_13_5: 0.5765177753422486
CV ROC AUC score without RVI_10: 0.5765177753422486
CV ROC AUC score without BBL_5_2.0_5: 0.5765177753422486
CV ROC AUC score without STOCHk_14_3_3: 0.5765177753422486
CV ROC AUC score without STOCHk_14_3_3_7_3_3: 0.5765177753422486
CV ROC AUC score without STOCHk_14_3_3_10_3_3: 0.5765177753422486
CV ROC AUC score without MACDh_6_13_5_6_13_5: 0.5765177753422486
CV ROC AUC score without MACDh_12_26_9: 0.5765177753422486
CV ROC AUC score without BBL_10_2.0_10: 0.5765177753422486
CV ROC AUC score without R3: 0.5765177753422486
CV ROC AUC score without R2: 0.5765177753422486
CV ROC AUC score without o_shift: 0.5765177753422486
CV ROC AUC score without R1: 0.5765177753422486
CV ROC AUC score without open: 0.5765177753422486
CV ROC AUC score without BBL_15_2.0_15: 0.5765177753422486
CV ROC AUC score without SMA_5: 0.5765177753422486
CV ROC AUC score wit

## Reverse Ablation ##

In [ ]:
# # Define initial features for ablation process
# ablation_features = [
#     "cmf",
#     "MACDs_6_13_5_6_13_5",
#     "RVI_10",
#     "BBL_5_2.0_5",
#     "STOCHk_14_3_3",
#     "STOCHk_14_3_3_7_3_3",
#     "STOCHk_14_3_3_10_3_3",
#     "MACDh_6_13_5_6_13_5",
#     "MACDh_12_26_9",
#     "BBL_10_2.0_10",
#     "R3",
#     "R2",
#     "o_shift",
#     "R1",
#     "open",
#     "BBL_15_2.0_15",
#     "SMA_5",
#     "BBM_5_2.0_5",
#     "c_shift",
#     "PP",
#     "BBL_20_2.0_20",
#     "EMA_5",
#     "ITS_5",
#     "SMA_10",
#     "BBM_10_2.0_10",
#     "EMA_10",
#     "S1",
#     "SMA_20",
#     "EMA_2",
#     "BBM_15_2.0_15",
#     "S2",
#     "BBM_20_2.0_20",
#     "IKS_15",
#     "close",
#     "PSARaf_0.01_0.1",
#     "S3",
#     "MACD_6_13_5_6_13_5",
#     "ISA_5",
#     "ISB_15",
#     "BBU_15_2.0_15",
#     "BBU_10_2.0_10",
#     "BBB_15_2.0_15",
#     "obv",
#     "BBU_5_2.0_5",
#     "high",
#     "STOCHd_14_3_3",
#     "STOCHd_14_3_3_7_3_3",
#     "STOCHd_14_3_3_10_3_3",
#     "TRIXs_10_5",
#     "MACD_12_26_9",
#     "PSARs_0.01_0.1",
#     "TRIXs_18_9",
#     "ICS_15",
#     "ROC_10",
#     "BBB_20_2.0_20",
#     "bollinger_bandwidth",
#     "RVI_15",
#     "MACDs_12_26_9",
#     "PSARl_0.01_0.1",
#     "TRIX_18_9",
#     "volume_diff_lag",
#     "TRIXs_12_6",
#     "ROC_5",
#     "BBU_20_2.0_20",
#     "low",
#     "TRIX_12_6",
#     "ROC_14",
# ]

# # All the features in the data
# all_features = list(df.columns)
# all_features.remove("color_change")  # we remove the target variable

# # Base features are all features excluding the ones in the ablation list
# base_features = [
#     feature for feature in all_features if feature not in ablation_features
# ]

# log_reg_scores = {}
# rf_scores = {}

# for feature in ablation_features:
#     # Add one feature from the ablation list
#     current_features = base_features + [feature]

#     # Prepare your features and target
#     X = df[current_features]
#     y = df["color_change"]

#     # Logistic Regression
#     log_reg = LogisticRegression(random_state=42, max_iter=500)

#     # Cross-validation
#     cv_scores = cross_val_score(log_reg, X, y, cv=tscv, scoring="roc_auc")
#     log_reg_scores[feature] = np.mean(cv_scores)

# # Print logistic regression scores sorted by score
# print("Logistic Regression Scores")
# for feature, score in sorted(
#     log_reg_scores.items(), key=lambda item: item[1], reverse=True
# ):
#     print(f"CV ROC AUC score with {feature}: {score}")

# for feature in ablation_features:
#     # Add one feature from the ablation list
#     current_features = base_features + [feature]

#     # Prepare your features and target
#     X = df[current_features]
#     y = df["color_change"]

#     # Random Forest
#     rf = RandomForestClassifier(n_estimators=100, random_state=42)

#     # Cross-validation
#     cv_scores = cross_val_score(rf, X, y, cv=tscv, scoring="roc_auc")
#     rf_scores[feature] = np.mean(cv_scores)

# # Print random forest scores sorted by score
# print("\nRandom Forest Scores")
# for feature, score in sorted(rf_scores.items(), key=lambda item: item[1], reverse=True):
#     print(f"CV ROC AUC score with {feature}: {score}")

## Feature Importance ##

In [ ]:
# X = df.drop("color_change", axis=1)
# y = df["color_change"]

# # Re-scale the data to include the new feature
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Fit XGBoost model and get feature importances
# xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss")
# xgb.fit(X_scaled, y)

# # Store the feature importances in a pandas series, then sort it in descending order
# importances = pd.Series(xgb.feature_importances_, index=X.columns)
# importances_sorted = importances.sort_values(ascending=False)

# print("Feature importances:")
# print(importances_sorted)

# # Now we can apply Logistic Regression and Random Forests using the features
# # Logistic Regression
# log_reg = LogisticRegression(random_state=42, max_iter=500)

# # Cross-validation
# cv_scores = cross_val_score(log_reg, X_scaled, y, cv=tscv, scoring="f1")

# print(f"\nLogistic Regression CV F1 score: {np.mean(cv_scores)}")

# # Random Forest
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Cross-validation
# cv_scores = cross_val_score(rf, X_scaled, y, cv=tscv, scoring="f1")

# print(f"Random Forest CV F1 score: {np.mean(cv_scores)}")

## Recursive Feature Elimination ##

In [ ]:
# X = df.drop("color_change", axis=1)
# y = df["color_change"]

# # Re-scale the data to include the new feature
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Recursive Feature Elimination
# # Here we use Logistic Regression as the model to evaluate the feature importance
# # You can replace it with any model you prefer
# model = LogisticRegression(random_state=42, max_iter=500)
# rfe = RFECV(model)
# rfe.fit(X_scaled, y)

# # Get the features ranking
# feature_ranking = {
#     feature_name: rank for feature_name, rank in zip(X.columns, rfe.ranking_)
# }

# # Sort and print the feature ranking
# for feature_name, rank in sorted(feature_ranking.items(), key=lambda item: item[1]):
#     print(f"{feature_name}: {rank}")

# # Transform X to include only the selected features
# X_transformed = rfe.transform(X_scaled)

# # Logistic Regression with transformed features
# log_reg = LogisticRegression(random_state=42, max_iter=500)

# # Cross-validation
# cv_scores = cross_val_score(log_reg, X_transformed, y, cv=tscv, scoring="f1")

# print(f"\nLogistic Regression CV F1 score: {np.mean(cv_scores)}")

# # Random Forest with transformed features
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Cross-validation
# cv_scores = cross_val_score(rf, X_transformed, y, cv=tscv, scoring="f1")

# print(f"Random Forest CV F1 score: {np.mean(cv_scores)}")

## Univariate Feature Selection Process ##

In [ ]:
# X = df.drop("color_change", axis=1)
# y = df["color_change"]

# # Re-scale the data to include the new feature
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # feature selection
# selector = SelectKBest(score_func=f_classif, k="all")
# selector.fit(X_scaled, y)

# # Get columns to keep and create new dataframe with those only
# cols = selector.get_support(indices=True)
# features_df_new = X.iloc[:, cols]

# # Store the scores of each feature in a dictionary
# feature_scores = {
#     feature_name: score for feature_name, score in zip(X.columns, selector.scores_)
# }

# # Sort the dictionary by value in descending order and print the scores
# for feature_name, score in sorted(
#     feature_scores.items(), key=lambda item: item[1], reverse=True
# ):
#     print(f"{feature_name}: {score}")

# # Now we can apply Logistic Regression and Random Forests on the new features_df_new
# # Logistic Regression
# log_reg = LogisticRegression(random_state=42, max_iter=500)

# # Cross-validation
# cv_scores = cross_val_score(log_reg, features_df_new, y, cv=tscv, scoring="f1")

# print(f"\nLogistic Regression CV F1 score: {np.mean(cv_scores)}")

# # Random Forest
# rf = RandomForestClassifier(n_estimators=100, random_state=42)

# # Cross-validation
# cv_scores = cross_val_score(rf, features_df_new, y, cv=tscv, scoring="f1")

# print(f"Random Forest CV F1 score: {np.mean(cv_scores)}")

# print("\n", features_df_new.columns)

## Base Model ##

In [ ]:
# Prepare your features and target
X = df.drop("color_change", axis=1)
y = df["color_change"]

# Logistic Regression
log_reg = LogisticRegression(random_state=42, max_iter=500)

# Cross-validation
cv_scores = cross_val_score(log_reg, X, y, cv=tscv, scoring="roc_auc")

print(f"Logistic Regression CV ROC AUC score: {np.mean(cv_scores)}")

# Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Cross-validation
cv_scores = cross_val_score(rf, X, y, cv=tscv, scoring="roc_auc")

print(f"Random Forest CV ROC AUC score: {np.mean(cv_scores)}")

print("\n", X.columns)

Logistic Regression CV ROC AUC score: 0.5765167194219271
Random Forest CV ROC AUC score: 0.8940717177408083

 Index(['time', 'volume', 'turnover', 'color', 'avg_vol_last_100', 'RSI_5',
       'RSI_10', 'RSI_14', 'ATR_14', 'ATR_10', 'ATR_5', 'CCI_14', 'CCI_10',
       'CCI_5', 'mfi', 'RVI_5', 'PSARr_0.01_0.1', 'TRIX_10_5', 'BBB_5_2.0_5',
       'BBP_5_2.0_5', 'BBB_10_2.0_10', 'BBP_10_2.0_10', 'BBP_15_2.0_15',
       'BBP_20_2.0_20', 'volume_diff', 'turnover_diff', 'close_open',
       'open_close', 'high_low', 'close_change', 'open_change', 'h_diff',
       'l_diff', 'hl_shift', 'high_pct', 'low_pct', 'close_open_lag',
       'high_low_lag'],
      dtype='object')
